<font size=6><b>펀드 운용모델
* 참고: 실습06_펀드_운용모델.xlsx

In [1]:
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr

from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time


import warnings
warnings.filterwarnings(action='ignore')

sns.set()
#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

# Data Load

## 시총 Top 5 업종명, 업종코드
* LG에너지솔루션, 삼성전자우 제외

In [2]:
start_data='20200101'
end_data='20221229'

df = stock.get_market_cap(end_data) 
top5_ticker = df.sort_values(by='시가총액', ascending=False).index[:7]
top5_name = []
for ticker in top5_ticker:
    names = stock.get_market_ticker_name(ticker)
    top5_name.append(names)
    print(ticker, names)
    time.sleep(1)  
    
top5_ticker = list(top5_ticker)

top5_ticker.remove('373220')
top5_ticker.remove('005935')

top5_name.remove('LG에너지솔루션')
top5_name.remove('삼성전자우')

print(len(top5_ticker), top5_ticker)
print(len(top5_name), top5_name)

005930 삼성전자
373220 LG에너지솔루션
207940 삼성바이오로직스
000660 SK하이닉스
051910 LG화학
005935 삼성전자우
006400 삼성SDI
5 ['005930', '207940', '000660', '051910', '006400']
5 ['삼성전자', '삼성바이오로직스', 'SK하이닉스', 'LG화학', '삼성SDI']


## 종목 기본정보
* 상장주식수, 유동비율
* fnguide 크롤링

In [3]:
from bs4 import BeautifulSoup
import requests
import re

In [4]:
발생주식수_list = []
유동비율_list = []
유동주식수_list = []

for ticker in top5_ticker:
    res = requests.get(f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{ticker}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=")
    soup = BeautifulSoup(res.text, 'html.parser')
    
    trdata = soup.select("#svdMainGrid1> table > tbody > tr")
    td = trdata[5].select("td")
    
    발행주식수 = td[0].text
    발행주식수 = 발행주식수.split("/")[0].strip()
    발행주식수 = int(''.join(re.findall('[\d.]+', 발행주식수)))
    
    유동비율 = td[1].text
    유동비율 = 유동비율.split('/')[1].strip()
    유동비율= float(유동비율)
    
    유동주식수 = td[1].text
    유동주식수 = 유동주식수.split('/')[0].strip()
    유동주식수 = int(''.join(re.findall('[\d.]+', 유동주식수)))
    
    발생주식수_list.append(발행주식수)
    유동비율_list.append(유동비율)
    유동주식수_list.append(유동주식수)
    
    print(발행주식수, 유동비율, 유동주식수)
    
    time.sleep(1)

5969782550 75.83 4526867390
71174000 24.76 17623981
728002365 73.83 537454512
70592343 66.63 47036786
68764530 73.16 50305248


In [5]:
발생주식수_list

[5969782550, 71174000, 728002365, 70592343, 68764530]

In [6]:
유동비율_list

[75.83, 24.76, 73.83, 66.63, 73.16]

In [7]:
유동주식수_list  

[4526867390, 17623981, 537454512, 47036786, 50305248]

## 일자별 주가

In [8]:
주가_list = []
for ticker in top5_ticker:
    names = stock.get_market_ticker_name(ticker)
    closev = stock.get_market_ohlcv(start_data, end_data, ticker)[['종가']]
    closev.columns = [names]
    주가_list.append(closev)
    time.sleep(1)  
    
주가_df = pd.concat(주가_list, axis=1)  
주가_df.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
주가_df.to_csv('./datasets/일자별주가.csv')

In [ ]:
# 좌수 = pd.read_csv('./datasets/실습07_설정환매_랜덤.csv', encoding='utf-8') 
# 좌수.columns = ['날짜','설정', '환매']
# 좌수 = 좌수.set_index('날짜')
# 좌수.head()

In [ ]:
주가_df = pd.read_csv('./datasets/실습07_설정환매_랜덤.csv', encoding='utf-8')
좌수.columns = ['날짜','설정', '환매']
주가_df = 주가_df.set_index("날짜")
주가_df.head()

# 일자별 시가 총액

In [ ]:
# 유동주식수 = np.array(발생주식수_list) * np.array(유동비율_list)*0.01
# 시가총액_df = 주가_df*유동주식수
시가총액_df = 주가_df*np.array(유동주식수_list)
시가총액_df.head()

In [ ]:
# 시가총액_df['합계'] = 시가총액_df.iloc[:,0]+시가총액_df.iloc[:,1]+시가총액_df.iloc[:,2]+시가총액_df.iloc[:,3]+시가총액_df.iloc[:,4]
시가총액_df['합계'] = 시가총액_df.sum(axis=1)
시가총액_df.head()

In [ ]:
시가총액_df['지수'] = 시가총액_df['합계']/시가총액_df.iloc[0,5]
시가총액_df.head()

# 펀드

## 편입비중

In [ ]:
편입비중_df = 시가총액_df[top5_name]
편입비중_df = 편입비중_df.div(시가총액_df['합계'], axis=0)
편입비중_df.head()

## 설정좌수, 환매좌수

In [ ]:
좌수 = pd.read_csv('./datasets/실습07_설정환매_랜덤.csv') 
# , parse_dates=['날짜']
좌수 = 좌수.set_index('날짜')
좌수.head()

In [ ]:
좌수['총좌수'] = (좌수['설정']-좌수['환매']).cumsum()
좌수.head()

## iNAV

In [ ]:
기준가 = 10000
iNAV = (시가총액_df['지수']/시가총액_df['지수'][0])*기준가
# iNAV = iNAV.astype(int)
iNAV

## AUM

In [ ]:
AUM = iNAV*좌수['총좌수']
AUM

## 일자별 매매내역

In [ ]:
주식잔고_df = 편입비중_df[top5_name]

주식잔고_df = 주식잔고_df.mul(AUM, axis=0)
주식잔고_df = 주식잔고_df.div(주가_df)

주식잔고_df = 주식잔고_df.astype(int)

주식잔고_df.head()

In [ ]:
매매내역_df = 주식잔고_df.diff()
매매내역_df.iloc[0,:] = 주식잔고_df.iloc[0,:]
매매내역_df.head()

## Total Fund Result

In [ ]:
fdf = pd.concat([편입비중_df,시가총액_df['지수'],  좌수, iNAV, AUM, 매매내역_df, 주식잔고_df], axis=1)
fdf.head()

In [ ]:
fdf.columns = ['편입비중_삼성전자', '편입비중_삼성바이오로직스', '편입비중_SK하이닉스', '편입비중_LG화학', '편입비중_삼성SDI',
              '지수', '설정좌수', '환매좌수', '총좌수', 'iNAV', 'AUM',
              '매매내역_삼성전자', '매매내역_삼성바이오로직스', '매매내역_SK하이닉스', '매매내역_LG화학', '매매내역_삼성SDI',
              '주식잔고_삼성전자', '주식잔고_삼성바이오로직스', '주식잔고_SK하이닉스', '주식잔고_LG화학', '주식잔고_삼성SDI']
fdf.head()

In [ ]:
fdf.tail()